In [15]:
import nltk
import ast
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet

_PosTaggedReviews= 'PosTaggedReviews.txt'
_Aspects= 'Aspects.txt'
_AspectSentiments= 'AspectSentiments.txt'
    
#Choosing the aspect from the POS tagging
def aspectExtraction(inputFileStr,outputFileStr,printResult):
    inputFile = open(inputFileStr,"r").read()
    outputFile=open (outputFileStr,"w")
    inputTupples=ast.literal_eval(inputFile)
    prevWord=''
    prevTag=''
    currWord=''
    aspectList=[]
    outputDict={}
    #Extracting Aspects
    for key,value in inputTupples.items():
        for word,tag in value:
            #choosing noun and noun phrases as the aspect
            if(tag=='NN' or tag=='NNP'):
                if(prevTag=='NN' or prevTag=='NNP'):
                    currWord= prevWord + ' ' + word
                else:
                    aspectList.append(prevWord.upper())
                    currWord= word
            prevWord=currWord
            prevTag=tag
    #Eliminating aspect which has 1 or less count
    for aspect in aspectList:
            if(aspectList.count(aspect)>1):
                    if(outputDict.keys()!=aspect):
                            outputDict[aspect]=aspectList.count(aspect)
    outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
    if(printResult):
        print(outputAspect)
    outputFile.write(str(outputAspect))
    outputFile.close()

#Function to find the sentiment of the aspect regarding to opinion data from task 2.1
def identifyOpinionWords(inputReviewListStr, inputAspectListStr, outputAspectOpinionListStr,printResult):       
    inputReviewList = open(inputReviewListStr,"r").read()
    inputAspectList = open(inputAspectListStr,"r").read()
    outputAspectOpinionList=open (outputAspectOpinionListStr,"w")
    inputReviewsTuples=ast.literal_eval(inputReviewList)
    inputAspectTuples=ast.literal_eval(inputAspectList)
    outputAspectOpinionTuples={}
    orientationCache={}
    negativeWordSet = {"don't","never", "nothing", "nowhere", "noone", "none", "not",
                  "hasn't","hadn't","can't","Can't","couldn't","shouldn't","won't",
                  "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't"}
    for aspect,no in inputAspectTuples:
        aspectTokens= word_tokenize(aspect)
        count=0
        for key,value in inputReviewsTuples.items():
            condition=True
            isNegativeSen=False
            for subWord in aspectTokens:
                if(subWord in str(value).upper()):
                    condition = condition and True
                else:
                    condition = condition and False
            if(condition):
                for negWord in negativeWordSet:
                    if(not isNegativeSen):#once senetence is negative no need to check this condition again and again
                        if negWord.upper() in str(value).upper():
                            isNegativeSen=isNegativeSen or True
                outputAspectOpinionTuples.setdefault(aspect,[0,0,0])
                for word,tag in value:
                     if(tag=='JJ' or tag=='JJR' or tag=='JJS'or tag== 'RB' or tag== 'RBR'or tag== 'RBS'):
                         count+=1
                         if(word not in orientationCache):
                             orien=orientation(word)
                             orientationCache[word]=orien
                         else:
                             orien=orientationCache[word]
                         if(isNegativeSen and orien is not None):
                             orien= not orien
                         if(orien==True):
                             outputAspectOpinionTuples[aspect][0]+=1
                         elif(orien==False):
                             outputAspectOpinionTuples[aspect][1]+=1
                         elif(orien is None):
                             outputAspectOpinionTuples[aspect][2]+=1
        if(count>0):
            #print(aspect,' ', outputAspectOpinionTuples[aspect][0], ' ',outputAspectOpinionTuples[aspect][1], ' ',outputAspectOpinionTuples[aspect][2])
            outputAspectOpinionTuples[aspect][0]=round((outputAspectOpinionTuples[aspect][0]/count)*100,2)
            outputAspectOpinionTuples[aspect][1]=round((outputAspectOpinionTuples[aspect][1]/count)*100,2)
            outputAspectOpinionTuples[aspect][2]=round((outputAspectOpinionTuples[aspect][2]/count)*100,2)
            print(aspect,':\t\tPositive => ', outputAspectOpinionTuples[aspect][0], '\tNegative => ',outputAspectOpinionTuples[aspect][1])
    if(printResult):
        print(outputAspectOpinionList)
    outputAspectOpinionList.write(str(outputAspectOpinionTuples))
    outputAspectOpinionList.close();
#-----------------------------------------------------------------------------------
def orientation(inputWord): 
    wordSynset=wordnet.synsets(inputWord)
    if(len(wordSynset) != 0):
        word=wordSynset[0].name()
        orientation=sentiwordnet.senti_synset(word)
        if(orientation.pos_score()>orientation.neg_score()):
            return True
        elif(orientation.pos_score()<orientation.neg_score()):
            return False
#    #   #
#To decide if the user want's to print the result        
def printResultChoice():
    userChoice = str(input('\nDo you want to print the result on output window? (Y/N) :'))
    if(userChoice=='Y' or userChoice=='y'):
        return True
    else:
        return False
        
# Calling the function
if __name__ == '__main__':
    print("\nNoun as Aspect")
    aspectExtraction(_PosTaggedReviews,_Aspects,printResultChoice())
    print("\nSentiments of aspects")
    identifyOpinionWords(_PosTaggedReviews,_Aspects,_AspectSentiments,printResultChoice())


Noun as Aspect

Do you want to print the result on output window? (Y/N) :y
[('QUALITY', 10), ('KIMONO', 7), ('PACKAGING', 6), ('BABY', 5), ('MAJORIE', 5), ('SERVICE', 5), ('THANK', 5), ('MESSAGE', 4), ('SHOP', 4), ('BOX', 3), ('DAUGHTER', 3), ('BOY', 3), ('LOVE', 3), ('HI MAJORIE', 3), ('EVERYONE', 3), ('ROMPER', 3), ('MATERIAL', 2), ('WORKMANSHIP', 2), ('PACKAGE', 2), ('ORDER', 2), ('OWNER', 2), ('HOPE', 2), ('KIDDO', 2), ('FABRIC', 2), ('NOTE', 2), ('FUTURE', 2), ('CUSTOMER SERVICE', 2), ('PIECE', 2)]

Sentiments of aspects

Do you want to print the result on output window? (Y/N) :y
QUALITY :		Positive =>  48.65 	Negative =>  8.11
KIMONO :		Positive =>  42.42 	Negative =>  18.18
PACKAGING :		Positive =>  17.65 	Negative =>  11.76
BABY :		Positive =>  38.1 	Negative =>  14.29
MAJORIE :		Positive =>  42.86 	Negative =>  0.0
SERVICE :		Positive =>  38.89 	Negative =>  5.56
THANK :		Positive =>  25.0 	Negative =>  5.0
MESSAGE :		Positive =>  54.55 	Negative =>  0.0
SHOP :		Positive =>  